### Assignment 7.1 A

In [5]:
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as parq

def get_key(val):
    for key, value in parts.items():
        #if val == str(value):
        if val in value:
            return key
    return "key doesn't exist"

partitions = (
        ('A', 'A'), ('B', 'B'), ('C', 'D'), ('E', 'F'),
        ('G', 'H'), ('I', 'J'), ('K', 'L'), ('M', 'M'),
        ('N', 'N'), ('O', 'P'), ('Q', 'R'), ('S', 'T'),
        ('U', 'U'), ('V', 'V'), ('W', 'X'), ('Y', 'Z')
    )
partitions_val_keys = (
        'A', 'B', 'C-D', 'E-F',
        'G-H', 'I-J', 'K-L', 'M',
        'N', 'O-P', 'Q-R', 'S-T',
        'U', 'V', 'W-X', 'Y-Z'
    )
parts = dict(zip(partitions_val_keys, partitions))
print(parts)

base_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07')
results_dir = base_dir.joinpath('results')
parquet_file = results_dir.joinpath('routes.parquet')
partitioned_parquet_file = results_dir.joinpath('kv')
csv_file = results_dir.joinpath('routes_parquet.csv')
data_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07').joinpath('Data')

pq = pd.read_parquet(parquet_file, engine='fastparquet')

print(list(pq.columns.values))

pq['key'] = pq['src_airport.iata']+pq['dst_airport.iata']+pq['airline.icao']
pq['partition_value'] = pq['key'].str[:1]
pq['kv_key'] = pq.apply(lambda x: get_key(x.partition_value), axis=1)

# Trying to remove bad keys
pq = pq[pq.kv_key != "key doesn't exist"]


table = pa.Table.from_pandas(pq)

parq.write_to_dataset(
    table,
    root_path=partitioned_parquet_file,
    partition_cols=['kv_key'],
)


ptable = parq.read_table(partitioned_parquet_file)
print(ptable)

{'A': ('A', 'A'), 'B': ('B', 'B'), 'C-D': ('C', 'D'), 'E-F': ('E', 'F'), 'G-H': ('G', 'H'), 'I-J': ('I', 'J'), 'K-L': ('K', 'L'), 'M': ('M', 'M'), 'N': ('N', 'N'), 'O-P': ('O', 'P'), 'Q-R': ('Q', 'R'), 'S-T': ('S', 'T'), 'U': ('U', 'U'), 'V': ('V', 'V'), 'W-X': ('W', 'X'), 'Y-Z': ('Y', 'Z')}
['codeshare', 'equipment', 'airline.active', 'airline.airline_id', 'airline.alias', 'airline.callsign', 'airline.country', 'airline.iata', 'airline.icao', 'airline.name', 'src_airport.airport_id', 'src_airport.altitude', 'src_airport.city', 'src_airport.country', 'src_airport.dst', 'src_airport.iata', 'src_airport.icao', 'src_airport.latitude', 'src_airport.longitude', 'src_airport.name', 'src_airport.source', 'src_airport.timezone', 'src_airport.type', 'src_airport.tz_id', 'dst_airport.airport_id', 'dst_airport.altitude', 'dst_airport.city', 'dst_airport.country', 'dst_airport.dst', 'dst_airport.iata', 'dst_airport.icao', 'dst_airport.latitude', 'dst_airport.longitude', 'dst_airport.name', 'dst_ai

### Assignment 7.1 B

In [6]:
import hashlib
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as parq

def hash_key(key):
    m = hashlib.sha256()
    m.update(str(key).encode('utf-8'))
    return m.hexdigest()

base_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07')
results_dir = base_dir.joinpath('results')
parquet_file = results_dir.joinpath('routes.parquet')
partitioned_parquet_file = results_dir.joinpath('hash')
csv_file = results_dir.joinpath('routes_parquet.csv')
data_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07').joinpath('Data')

pq = pd.read_parquet(parquet_file, engine='fastparquet')

pq['key'] = pq['src_airport.iata']+pq['dst_airport.iata']+pq['airline.icao']
pq['hashed'] = pq.apply(lambda x: hash_key(x.key), axis=1)
pq['hash_key'] = pq['hashed'].str[:1]


table = pa.Table.from_pandas(pq)

parq.write_to_dataset(
    table,
    root_path=partitioned_parquet_file,
    partition_cols=['hash_key'],
)

ptable = parq.read_table(partitioned_parquet_file)
print(ptable)

pyarrow.Table
codeshare: bool
equipment: list<item: string>
  child 0, item: string
airline.active: bool
airline.airline_id: int64
airline.alias: string
airline.callsign: string
airline.country: string
airline.iata: string
airline.icao: string
airline.name: string
src_airport.airport_id: double
src_airport.altitude: double
src_airport.city: string
src_airport.country: string
src_airport.dst: string
src_airport.iata: string
src_airport.icao: string
src_airport.latitude: double
src_airport.longitude: double
src_airport.name: string
src_airport.source: string
src_airport.timezone: double
src_airport.type: string
src_airport.tz_id: string
dst_airport.airport_id: double
dst_airport.altitude: double
dst_airport.city: string
dst_airport.country: string
dst_airport.dst: string
dst_airport.iata: string
dst_airport.icao: string
dst_airport.latitude: double
dst_airport.longitude: double
dst_airport.name: string
dst_airport.source: string
dst_airport.timezone: double
dst_airport.type: string
dst_a

### Assignment 7.1 C

In [7]:
import pygeohash
import pandas as pd
from pathlib import Path
import pyarrow as pa
import pyarrow.parquet as parq


def get_data_center_val(lat, long, westval, centralval, eastval):
    def get_center(val):
        for key, value in parts.items():
            if val == value:
                return key
        return "key doesn't exist"

    values = []
    locations = ['west','central','east']
    srcgeoval = pygeohash.encode(lat, long)
    distm_west = pygeohash.geohash_approximate_distance(srcgeoval, westval) / 1000
    values.append(distm_west)
    distm_central = pygeohash.geohash_approximate_distance(srcgeoval, centralval) / 1000
    values.append(distm_central)
    distm_east = pygeohash.geohash_approximate_distance(srcgeoval, eastval) / 1000
    values.append(distm_east)
    parts = dict(zip(locations, values))
    closestdist =  min(values, key=float)
    center = get_center(closestdist)
    return center

westval = pygeohash.encode(45.5945645, -121.1786823, precision=12)
print(westval)
centralval = pygeohash.encode(41.1544433, -96.0422378, precision=12)
print(centralval)
eastval = pygeohash.encode(39.08344, -77.6497145, precision=12)
print(eastval)

base_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07')
results_dir = base_dir.joinpath('results')
parquet_file = results_dir.joinpath('routes.parquet')
partitioned_parquet_file = results_dir.joinpath('geo')
csv_file = results_dir.joinpath('routes_parquet_datacenters.csv')
data_dir = Path('C:/Users/chris/MyNewJNFolder/dsc650/assignments/assignment07').joinpath('Data')

pq = pd.read_parquet(parquet_file, engine='fastparquet')

pq['key'] = pq['src_airport.iata']+pq['dst_airport.iata']+pq['airline.icao']
pq['lat'] = pq['src_airport.latitude']
pq['long'] = pq['src_airport.longitude']
pq['location'] = pq.apply(lambda x: get_data_center_val(x.lat, x.long, westval, centralval, eastval), axis=1)


table = pa.Table.from_pandas(pq)

parq.write_to_dataset(
    table,
    root_path=partitioned_parquet_file,
    partition_cols=['location'],
)

ptable = parq.read_table(partitioned_parquet_file)
print(ptable)

c21g6s0rs4c7
9z7dnebnj8kb
dqby34cjw922
pyarrow.Table
codeshare: bool
equipment: list<item: string>
  child 0, item: string
airline.active: bool
airline.airline_id: int64
airline.alias: string
airline.callsign: string
airline.country: string
airline.iata: string
airline.icao: string
airline.name: string
src_airport.airport_id: double
src_airport.altitude: double
src_airport.city: string
src_airport.country: string
src_airport.dst: string
src_airport.iata: string
src_airport.icao: string
src_airport.latitude: double
src_airport.longitude: double
src_airport.name: string
src_airport.source: string
src_airport.timezone: double
src_airport.type: string
src_airport.tz_id: string
dst_airport.airport_id: double
dst_airport.altitude: double
dst_airport.city: string
dst_airport.country: string
dst_airport.dst: string
dst_airport.iata: string
dst_airport.icao: string
dst_airport.latitude: double
dst_airport.longitude: double
dst_airport.name: string
dst_airport.source: string
dst_airport.timezone

### Assignment 7.1 D

In [8]:
def balance_partitions (keys, num_partitions):
    uniquevals = sorted(set(keys))
    num_uniq_vals = len(uniquevals)
    partition_counts = (num_uniq_vals / num_partitions)+1
    partitions  = []
    currow = 1
    partnum = 1
    for i in range(num_uniq_vals):
        curkeyval ={}
        if currow <= partition_counts:
            curkeyval[uniquevals[i]] = partnum
            currow = currow + 1
        else:
            currow = 1
            partnum = partnum + 1
            curkeyval[uniquevals[i]] = partnum
            currow = currow + 1
        partitions .append(curkeyval)
    return partitions


keys = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6', 'test7', 'test8', 'test9', 'test10', 'test11']
num_partitions = 3
partitions  = balance_partitions(keys, num_partitions)
print(type(partitions ))
print(partitions )

keys = ['test1', 'test2', 'test3', 'test4', 'test5', 'test6', 'test7', 'test8', 'test9', 'test10', 'test11', 'test12', 'test13', 'test14', 'test15', 'test16']
num_partitions = 3
partitions  = balance_partitions(keys, num_partitions)
print(type(partitions ))
print(partitions )

<class 'list'>
[{'test1': 1}, {'test10': 1}, {'test11': 1}, {'test2': 1}, {'test3': 2}, {'test4': 2}, {'test5': 2}, {'test6': 2}, {'test7': 3}, {'test8': 3}, {'test9': 3}]
<class 'list'>
[{'test1': 1}, {'test10': 1}, {'test11': 1}, {'test12': 1}, {'test13': 1}, {'test14': 1}, {'test15': 2}, {'test16': 2}, {'test2': 2}, {'test3': 2}, {'test4': 2}, {'test5': 2}, {'test6': 3}, {'test7': 3}, {'test8': 3}, {'test9': 3}]
